In [285]:
!pip install slpp

In [ ]:
import json
import logging
import re
import tempfile
from collections import defaultdict
from dataclasses import dataclass, field
from io import BytesIO
from pathlib import Path
from urllib.request import urlopen
from zipfile import ZipFile

from graphviz import Digraph
from PIL import Image, ImageDraw, ImageFont, ImageFilter
from slpp import slpp as lua

logger = logging.getLogger(__name__)

# Configs

In [ ]:
@dataclass(frozen=True)
class PathConfig:
    """File and directory paths."""

    output_dir: Path = Path("output")

    # Factorio data files
    recipe_file: str = "base/prototypes/recipe.lua"
    item_file: str = "base/prototypes/item.lua"
    fluid_file: str = "base/prototypes/fluid.lua"
    equipment_file: str = "base/prototypes/equipment.lua"


@dataclass(frozen=True)
class MachineConfig:
    """Machine types and their speeds."""

    crafting_machine: str = "assembling-machine-3"
    smelting_machine: str = "electric-furnace"

    @property
    def crafting_speed(self) -> float:
        speeds = {
            "assembling-machine-1": 0.5,
            "assembling-machine-2": 0.75,
            "assembling-machine-3": 1.25,
        }
        return speeds.get(self.crafting_machine, 1.25)

    @property
    def smelting_speed(self) -> float:
        speeds = {
            "stone-furnace": 1.0,
            "steel-furnace": 2.0,
            "electric-furnace": 2.0,
        }
        return speeds.get(self.smelting_machine, 2.0)

    @property
    def recipe_speeds(self) -> dict[str, float]:
        """Get speeds for all recipe categories."""
        return {
            "advanced-crafting": self.crafting_speed,
            "crafting": self.crafting_speed,
            "crafting-with-fluid": self.crafting_speed,
            "smelting": self.smelting_speed,
        }

    @property
    def by_category(self) -> dict[str, str]:
        return {
            "advanced-crafting": self.crafting_machine,
            "crafting": self.crafting_machine,
            "crafting-with-fluid": self.crafting_machine,
            "smelting": self.smelting_machine,
            "oil-processing": "oil-refinery",
            "chemistry": "chemical-plant",
            "rocket-building": "rocket-silo",
            "centrifuging": "centrifuge",
        }


@dataclass(frozen=True)
class GameConfig:
    """Factorio game settings."""

    version: str = "2.0.71"
    difficulty: str = "normal"

    @property
    def data_github_url(self) -> str:
        return f"https://raw.githubusercontent.com/wube/factorio-data/refs/tags/{self.version}"

    @property
    def icon_github_url(self) -> str:
        return "https://raw.githubusercontent.com/deniszholob/icons-factorio/main/factorio-icons/base/icons"


@dataclass(frozen=True)
class Config:
    """Master configuration."""

    paths: PathConfig = PathConfig()
    machines: MachineConfig = MachineConfig()
    game: GameConfig = GameConfig()


# Create global config instance
config = Config()
config.paths.output_dir.mkdir(exist_ok=True)

In [ ]:
def read_factorio_file(file_path: str) -> str:
    """
    Returns the contents of a Factorio data file from GitHub.

    Args:
        file_path: relative path inside Factorio folder
    """
    file_path = file_path.replace("__base__", "base")
    url = f"{config.game.data_github_url}/{file_path}"
    with urlopen(url) as response:
        data = response.read()
        return data.decode("utf-8")


def clean_factorio_lua(text: str) -> str:
    """Clean Factorio Lua file by removing comments and unnecessary parts."""
    # Remove multi-line comments
    text = re.sub(r"--\[\[.*?\]\]", "", text, flags=re.DOTALL)
    # Remove single-line comments
    text = re.sub(r"--.*", "", text)
    return text


def format_number(value: float, max_decimals: int = 3) -> str:
    """Format number with up to max_decimals places, no trailing zeros."""
    return f"{value:.{max_decimals}f}".rstrip("0").rstrip(".")

# Clean recipes

In [ ]:
@dataclass
class Recipe:
    name: str
    category: str = "crafting"
    energy_required: float = 0.5
    ingredients: dict[str, int] = field(default_factory=dict)
    result_amount: float = 1

    # derived values
    machine: str = field(init=False)
    time_per_cycle: float = field(init=False)

    def __post_init__(self):
        speeds = config.machines.recipe_speeds
        default_speed = config.machines.crafting_speed
        self.time_per_cycle = self.energy_required / speeds.get(
            self.category, default_speed
        )

        self.machine = config.machines.by_category.get(self.category)
        if self.machine is None:
            logger.warning(
                "Couldn't determine name for recipe %s with category %s",
                self.name,
                self.category,
            )

In [ ]:
recipe_file_text = read_factorio_file(config.paths.recipe_file)

# Remove wrapper
text = recipe_file_text.split("data:extend\n(", 1)[1].removesuffix(")")
text = clean_factorio_lua(text)
raw_recipes = lua.decode(text)

(config.paths.output_dir / "recipe.json").write_text(json.dumps(raw_recipes, indent=2))

logger.info("Number of raw recipes found: %s", len(raw_recipes))


In [ ]:
def parse_recipe(raw_recipe: dict) -> Recipe | None:
    """Parse a raw Factorio recipe dict into a Recipe object.

    Returns None if the recipe should be skipped (e.g., multiple results).
    """
    try:
        name = raw_recipe["name"]
        amount_dict = raw_recipe.get(config.game.difficulty, raw_recipe)

        # Parse ingredients safely
        raw_ingredients = amount_dict.get("ingredients", [])
        ingredients: dict[str, int] = {
            ing["name"]: ing["amount"]
            for ing in raw_ingredients
            if "name" in ing and "amount" in ing
        }

        # Parse results safely
        raw_results = amount_dict.get("results", [])
        if len(raw_results) != 1:
            logger.warning("Skipping %s: multiple results", name)
            return None

        result_dict = next(iter(raw_results))
        result_name = result_dict.get("name")
        result_amount = result_dict.get("amount")

        if result_name != name:
            logger.warning("Skipping %s: result name differs (%s)", name, result_name)
            return None

        # Determine category and energy
        category = raw_recipe.get("category", "crafting")
        energy_required = amount_dict.get("energy_required", 0.5)

        if category not in config.machines.by_category:
            logger.warning("Unknown recipe category '%s' for recipe %s", category, name)

        return Recipe(
            name=name,
            category=category,
            ingredients=ingredients,
            energy_required=energy_required,
            result_amount=result_amount,
        )

    except Exception as e:
        logger.error("Failed to parse recipe %s: %s", raw_recipe.get("name"), e)
        return None


# Usage: parse all raw recipes
recipes: dict[str, Recipe] = {}

for raw_recipe in raw_recipes:
    recipe = parse_recipe(raw_recipe)
    if recipe is not None:
        recipes[recipe.name] = recipe


# Icon Management

In [ ]:
class IconManager:
    """Manages loading, caching, and rendering of Factorio icons."""

    FONT_URL = "https://github.com/dejavu-fonts/dejavu-fonts/releases/download/version_2_37/dejavu-fonts-ttf-2.37.zip"
    FONT_NAME = "DejaVuSans-Bold.ttf"

    def __init__(self, font_size: int = 24):
        self._cache: dict[tuple[str, int], Image.Image] = {}
        self.font = self._load_font(font_size)
        self.icon_paths = self._load_all_icon_paths()


    def _load_font(self, font_size: int) -> ImageFont.FreeTypeFont:
        """Load the font, downloading if necessary."""
        with urlopen(self.FONT_URL) as resp:
            zip_bytes = BytesIO(resp.read())

        with ZipFile(zip_bytes) as zf:
            for name in zf.namelist():
                if not name.endswith(self.FONT_NAME):
                    continue
                with zf.open(name) as font_file:
                    font_bytes = BytesIO(font_file.read())
                return ImageFont.truetype(font_bytes, size=font_size)

        raise RuntimeError(f"{self.FONT_NAME} not found in zip!")

    def _load_all_icon_paths(self) -> dict[str, str]:
        """Load all icon paths from item, fluid, and equipment files."""
        paths = {}

        # Items
        item_text = read_factorio_file(config.paths.item_file)
        item_text = item_text.split("data:extend\n(", 1)[1].removesuffix(")\n")
        raw_items = lua.decode(clean_factorio_lua(item_text))
        paths.update({item.get("name"): item.get("icon") for item in raw_items})

        # Fluids
        fluid_text = read_factorio_file(config.paths.fluid_file)
        fluid_text = fluid_text.split("data:extend(", 2)[2].removesuffix(")\n")
        raw_fluids = lua.decode(clean_factorio_lua(fluid_text))
        paths.update({fluid.get("name"): fluid.get("icon") for fluid in raw_fluids})

        # Equipment
        equip_text = read_factorio_file(config.paths.equipment_file)
        equip_text = equip_text.split("data:extend(", 1)[1].removesuffix(")\n")
        raw_equipment = lua.decode(clean_factorio_lua(equip_text))
        paths.update({eq.get("name"): eq.get("icon") for eq in raw_equipment})

        return paths

    def load_icon(self, name: str) -> Image.Image | None:
        """Load icon from GitHub with caching."""
        if name in self._cache:
            return self._cache[name]

        icon_path = self.icon_paths[name]

        parts = icon_path.split("icons/", 1)
        if len(parts) != 2:
            logger.warning("Couldn't find icon path for %s", name)
            return None
        rel_path = parts[1]
        url = f"{config.game.icon_github_url}/{rel_path}"

        try:
            with urlopen(url) as response:
                img = Image.open(BytesIO(response.read())).convert("RGBA")
                self._cache[name] = img
                return img
        except Exception:
            logger.warning("Failed to load icon for %s", name)
            return None

    def create_shadow(
        self,
        product_img: Image.Image,
        canvas_size: tuple[int, int],
        position: tuple[int, int],
        intensity: float = 2.5,
        blur_radius: int = 6,
        expand_size: int = 5,
    ) -> Image.Image:
        """Create a drop shadow for an icon."""
        shadow = Image.new("RGBA", product_img.size, (0, 0, 0, 0))
        shadow.putalpha(product_img.split()[3])

        shadow_padded = Image.new("RGBA", canvas_size, (0, 0, 0, 0))
        shadow_padded.paste(shadow, position)
        shadow_padded = shadow_padded.filter(ImageFilter.MaxFilter(size=expand_size))
        shadow_padded = shadow_padded.filter(
            ImageFilter.GaussianBlur(radius=blur_radius)
        )

        alpha = shadow_padded.split()[3].point(lambda p: min(int(p * intensity), 255))
        shadow_padded.putalpha(alpha)

        return shadow_padded

    def add_text_overlay(
        self,
        image: Image.Image,
        top_right_text: str,
        bottom_right_text: str,
        stroke_width: int = 3,
    ) -> Image.Image:
        """Add text overlays to top-right and bottom-right corners."""
        draw = ImageDraw.Draw(image)
        w, h = image.size

        # Top-right text (production rate)
        if top_right_text:
            bbox = draw.textbbox((0, 0), top_right_text, font=self.font)
            text_w = bbox[2] - bbox[0]
            draw.text(
                (w - stroke_width - text_w, 0),
                top_right_text,
                fill="white",
                font=self.font,
                stroke_width=stroke_width,
                stroke_fill="black",
            )

        # Bottom-right text (machine count)
        if bottom_right_text:
            bbox = draw.textbbox((0, 0), bottom_right_text, font=self.font)
            text_w = bbox[2] - bbox[0]
            text_h = bbox[3] - bbox[1]
            draw.text(
                (w - stroke_width - text_w, h - 10 - text_h),
                bottom_right_text,
                fill="white",
                font=self.font,
                stroke_width=stroke_width,
                stroke_fill="black",
            )

        return image

    def create_product_image(
        self,
        product: str,
        machine: str | None,
        production_rate: float,
        machine_count: float,
        save_dir: Path,
        filename: str | None = None,
    ) -> str | None:
        """Create a complete product image with machine, shadow, and text overlays."""
        product_img = self.load_icon(product)
        if product_img is None:
            return None

        if machine_count and machine:
            machine_img = self.load_icon(machine)
            if machine_img is None:
                return None

            # Create base image
            image = machine_img.resize((128, 128), resample=Image.BICUBIC)

            # Add shadow
            shadow = self.create_shadow(
                product_img,
                canvas_size=image.size,
                position=(32, 32),
            )
            image = Image.alpha_composite(image, shadow)

            # Add product on top
            image.paste(product_img, (32, 32), product_img)

            machine_text = f"{format_number(machine_count)}"
        else:
            image = product_img.resize((128, 128), resample=Image.BICUBIC)
            machine_text = ""

        # Add text overlays
        production_text = f"{format_number(production_rate)}/s"
        image = self.add_text_overlay(image, production_text, machine_text)

        if filename is None:
            filename = f"{product}_with_{machine}_{format_number(production_rate)}_{format_number(machine_count)}.png"

        save_path = save_dir / filename

        # Save
        image.save(save_path)
        return str(save_path)


# Usage:
icon_manager = IconManager()
icon_path = icon_manager.create_product_image(
    product="automation-science-pack",
    machine="assembling-machine-3",
    production_rate=12.5,
    machine_count=10,
    save_dir=config.paths.output_dir,
)

# Build Graph

In [ ]:
@dataclass
class ProductionNode:
    """Represents a product in the production graph."""

    product: str
    recipe: Recipe | None
    item_demand: float  # items per second
    machine_count: float  # number of machines needed


@dataclass
class ProductionEdge:
    """Represents a connection between two products."""

    ingredient: str
    product: str
    item_flow: float  # items per second flowing along this edge
    machine_count: float  # machines needed to produce this flow


class GraphBuilder:
    """Builds and analyzes Factorio production graphs."""

    def __init__(self, recipes: dict[str, Recipe]):
        self.recipes = recipes

    def calculate_production_graph(
        self,
        target: str,
        target_demand: float = 2.5,
        raw_materials: set[str] = None,
    ) -> tuple[dict[str, ProductionNode], list[ProductionEdge]]:
        """
        Calculate production requirements for a target product.

        Returns:
            nodes: dict mapping product name to ProductionNode
            edges: list of ProductionEdges showing material flow
        """
        raw_materials = raw_materials or set()

        # Get topological ordering
        topo_order = self._get_topological_order(target, raw_materials)

        # Initialize target node
        target_recipe = self.recipes[target]
        item_demands = defaultdict(float, {target: target_demand})
        machine_counts = defaultdict(
            float,
            {
                target: target_demand
                / target_recipe.result_amount
                * target_recipe.time_per_cycle
            },
        )

        # Calculate demands backwards through graph
        edges: list[ProductionEdge] = []
        for product in reversed(topo_order):
            recipe = self.recipes.get(product)
            if product in raw_materials or recipe is None:
                continue  # raw material, no inputs

            for ingredient in recipe.ingredients:
                # Calculate flow along this edge
                item_flow = (
                    machine_counts[product]
                    * recipe.ingredients[ingredient]
                    / recipe.time_per_cycle
                )
                item_demands[ingredient] += item_flow

                # Calculate machines needed for ingredient
                machine_count_edge = 0.0
                ing_recipe = self.recipes.get(ingredient)
                if ing_recipe:
                    machine_count_edge = (
                        item_flow / ing_recipe.result_amount * ing_recipe.time_per_cycle
                    )
                machine_counts[ingredient] += machine_count_edge

                edges.append(
                    ProductionEdge(
                        ingredient=ingredient,
                        product=product,
                        item_flow=item_flow,
                        machine_count=machine_count_edge,
                    )
                )

        # Build nodes dict
        nodes = {
            product: ProductionNode(
                product=product,
                recipe=self.recipes.get(product),
                item_demand=item_demands[product],
                machine_count=machine_counts.get(product, 0.0),
            )
            for product in topo_order
        }

        return nodes, edges

    def _get_topological_order(self, target: str, raw_materials: set[str]) -> list[str]:
        """Get a topological ordering of products via DFS."""
        visited = set()
        topo_order = []

        def dfs(product: str):
            if product in visited:
                return
            visited.add(product)
            recipe = self.recipes.get(product)
            is_raw_material = product in raw_materials or recipe is None
            if not is_raw_material:
                for ingredient in recipe.ingredients:
                    dfs(ingredient)
            topo_order.append(product)

        dfs(target)
        return topo_order

    def visualize(
        self,
        nodes: dict[str, ProductionNode],
        edges: list[ProductionEdge],
        icon_manager: IconManager,
        raw_materials: set[str],
        output_path: Path,
    ) -> None:
        """Create a Graphviz visualization of the production graph."""
        dot = Digraph()

        with tempfile.TemporaryDirectory() as tmpdir:
            # Create node images
            for product, node in nodes.items():
                machine = None
                if node.recipe and product not in raw_materials:
                    machine = node.recipe.machine
                icon_path = icon_manager.create_product_image(
                    product=node.product,
                    machine=machine,
                    production_rate=node.item_demand,
                    machine_count=node.machine_count,
                    save_dir=Path(tmpdir),
                )
                dot.node(
                    product,
                    label="",
                    image=str(icon_path),
                    shape="none",
                    imagescale="true"
                )

            # Create edges
            for edge in edges:
                label = f"{format_number(edge.item_flow)}/s"
                dot.edge(edge.ingredient, edge.product, label=label)

            dot.render(str(output_path), format="png", cleanup=True)

    def build_and_visualize(
        self,
        target: str,
        icon_manager: IconManager,
        target_demand: float = 2.5,
        raw_materials: set[str] = set(),
        output_path: Path = None,
    ) -> tuple[dict[str, ProductionNode], list[ProductionEdge]]:
        """Convenience method to calculate and visualize in one call."""
        nodes, edges = self.calculate_production_graph(
            target, target_demand, raw_materials
        )

        if output_path is None:
            output_path = config.paths.output_dir / target

        self.visualize(nodes, edges, icon_manager, raw_materials, output_path)
        return nodes, edges


# Usage:
icon_manager = IconManager()
graph_builder = GraphBuilder(recipes)

# Simple usage
graph_builder.build_and_visualize(
    "electronic-circuit", icon_manager,
)


In [ ]:
raw_materials = {
    "iron-plate",
    "copper-plate",
    "steel-plate",
    "electronic-circuit",
    "advanced-circuit",
    "processing-unit",
    "sulfur",
    "plastic-bar",
    "stone-brick",
    "sulfuric-acid"
}
icon_manager = IconManager()
graph_builder = GraphBuilder(recipes)
for product in [
    "automation-science-pack",
    "logistic-science-pack",
    "chemical-science-pack",
    "military-science-pack",
    "production-science-pack",
    "utility-science-pack",
]:
    graph_builder.build_and_visualize(
        product,
        icon_manager,
        raw_materials=raw_materials,
    )
